## Code Movement

### Move code out of loop

In [11]:
%%timeit
accum=0
for i in range(1000000):
    item=10
    accum+=item

45.1 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
accum=0
item=10
for i in range(1000000):
    accum+=item

40.6 ms ± 792 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


A 10% speedup

### Move code out of loop header

#### Test 0: Preamble

Build a long list to test with

In [31]:
mylist=[]
for i in range(1000000):
    mylist.append(i)

In [55]:
len(mylist)

1000000

#### Test 1. Run the version where len(mylist) is called each loop.

In [51]:
%%timeit
accum=0
i=0
while i<len(mylist):
    accum+=1
    i+=1
    

117 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Test 2. Run the version where len(mylist) is called once and referred to each loop

In [52]:
%%timeit
accum=0
mylen=len(mylist)
i=0
while i<mylen:
    accum+=1
    i+=1

60.6 ms ± 385 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


This nearly halves the runtime of the loop. Clearly this is a massive improvement. What else could we do to improve this code.

#### Test 3. Use a for loop rather than a while loop (Note the a+=1 is left in to ensure the same number of operations per loop).

In [62]:
%%timeit
accum=0
mylen=len(mylist) #dead code
a=0               #dead code
for i in range(len(mylist)):
    accum+=1
    a+=1          #dead code

62.6 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Note that the While loop is faster that the for loop.

#### Test 4. Use a for loop but remove the incrementer and other dead code.

In [63]:
%%timeit
accum=0
for i in range(len(mylist)):
    accum+=1

40.4 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


A logical speedup due to the halving of the number of operations in this calculation.

#### Test 5. Move the len(mylist) call out of the loop header.

In [64]:
%%timeit
accum=0
mylen=len(mylist)
for i in range(mylen):
    accum+=1

40.6 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


No Change in speed.

## Strength Reduction

### Change multiplications to additions

Test 1. Convert a doubling to addition

In [73]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i*2

66 ms ± 926 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i+i

63.8 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Addition seems much faster. Does this hold for higher level multiplications?

Test 2. Convert a quadrupling to addition

In [75]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i*4

65 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [76]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i+i+i+i

106 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Loss of performance. Perhaps this is only works for multiplication by 2.

Test 3. Bit shift operations. You can replication multiplication by powers of two using the bit shift operator (<<), so which is faster?


In [77]:
a=99
print(a*2)
print(a<<1)

198
198


In [78]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i*2

65.2 ms ± 277 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i+i

63.5 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [80]:
%%timeit
accum=0
for i in range(1000000):
    accum+=i<<2

71.2 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Paradoxically, bit shift operations seem slower. Thank god, because they look horrible to read!

### 


## Loop Fusion

Combine two identical loops into one loop.

In [81]:
%%timeit
accum1=0
accum2=0
for i in range(1000000):
    accum1+=1
for i in range(1000000):
    accum2+=1

82.5 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [82]:
%%timeit
accum1=0
accum2=0
for i in range(1000000):
    accum1+=1
    accum2+=1

66.3 ms ± 708 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Unswitching

If a loop can be split into two simpler loops with logic outside do it.

In [85]:
%%timeit
mylist1=[]
mylist2=[]
X=1
Y=2

for i in range(1000000):
    if X <= Y:
        mylist1.append(i)
    else:
        mylist2.append(i)

85.6 ms ± 579 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
%%timeit
mylist1=[]
mylist2=[]
X=1
Y=2

if X <= Y:
    for i in range(1000000):
        mylist1.append(i)
else:
    for i in range(1000000):
        mylist2.append(i)

69.3 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Array Linearisation

In [90]:
%%timeit
mylist=[]
for i in range(20):
    for j in range(20):
        mylist.append(0)

25.3 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [91]:
%%timeit
mylist=[]
for i in range(400):
    mylist.append(0)

22.7 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Loop Unrolling

In [97]:
%%timeit
lista=[i for i in   range(1000000)]
listb=[j*2 for j in range(1000000)]
listc=[0 for k in   range(1000000)]
for i in range(1000000):
    listc[i]=lista[i]+listb[i]
    

205 ms ± 2.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%%timeit
lista=[i for i in   range(1000000)]
listb=[j*2 for j in range(1000000)]
listc=[0 for k in   range(1000000)]
for i in range(0,1000000,2):
    listc[i]=lista[i]+listb[i]
    listc[i+1]=lista[i+1]+listb[i+1]

226 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%timeit
lista=[i for i in   range(1000000)]
listb=[j*2 for j in range(1000000)]
listc=[0 for k in   range(1000000)]
for i in range(0,1000000,4):
    listc[i]=lista[i]+listb[i]
    listc[i+1]=lista[i+1]+listb[i+1]
    listc[i+2]=lista[i+2]+listb[i+2]
    listc[i+3]=lista[i+3]+listb[i+3]

245 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
%%timeit
lista=[i for i in   range(10)]
listb=[j*2 for j in range(10)]
listc=[0 for k in   range(10)]
for i in range(10):
    listc[i]=lista[i]+listb[i]
    

2.6 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [102]:
%%timeit
lista=[i for i in   range(10)]
listb=[j*2 for j in range(10)]
listc=[0 for k in   range(10)]

listc[0]=lista[0]+listb[0]
listc[1]=lista[1]+listb[1]
listc[2]=lista[2]+listb[2]
listc[3]=lista[3]+listb[3]
listc[4]=lista[4]+listb[4]
listc[5]=lista[5]+listb[5]
listc[6]=lista[6]+listb[6]
listc[7]=lista[7]+listb[7]
listc[8]=lista[8]+listb[8]
listc[9]=lista[9]+listb[9]

2.33 µs ± 34.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [100]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9
